Monocle2 functions
1. reduceDimension

In [ ]:
function (cds, max_components = 2, reduction_method = c("DDRTree", 
                                                        "ICA", "tSNE", "SimplePPT", "L1-graph", 
                                                        "SGL-tree"), norm_method = c("log", "vstExprs", 
                                                                                     "none"), residualModelFormulaStr = NULL, pseudo_expr = 1, 
          relative_expr = TRUE, auto_param_selection = TRUE, verbose = FALSE, 
          scaling = TRUE, ...) 
{
  extra_arguments <- list(...)
  set.seed(2016)
  FM <- normalize_expr_data(cds, norm_method, pseudo_expr)
  xm <- Matrix::rowMeans(FM)
  xsd <- sqrt(Matrix::rowMeans((FM - xm)^2))
  FM <- FM[xsd > 0, ]
  if (is.null(residualModelFormulaStr) == FALSE) {
    if (verbose) 
      message("Removing batch effects")
    X.model_mat <- sparse.model.matrix(as.formula(residualModelFormulaStr), 
                                       data = pData(cds), drop.unused.levels = TRUE)
    fit <- limma::lmFit(FM, X.model_mat, ...)
    beta <- fit$coefficients[, -1, drop = FALSE]
    beta[is.na(beta)] <- 0
    FM <- as.matrix(FM) - beta %*% t(X.model_mat[, -1])
  }
  else {
    X.model_mat <- NULL
  }
  if (scaling) {
    FM <- as.matrix(Matrix::t(scale(Matrix::t(FM))))
    FM <- FM[!is.na(row.names(FM)), ]
  }
  else FM <- as.matrix(FM)
  if (nrow(FM) == 0) {
    stop("Error: all rows have standard deviation zero")
  }
  FM <- FM[apply(FM, 1, function(x) all(is.finite(x))), ]
  if (is.function(reduction_method)) {
    reducedDim <- reduction_method(FM, ...)
    colnames(reducedDim) <- colnames(FM)
    reducedDimW(cds) <- as.matrix(reducedDim)
    reducedDimA(cds) <- as.matrix(reducedDim)
    reducedDimS(cds) <- as.matrix(reducedDim)
    reducedDimK(cds) <- as.matrix(reducedDim)
    dp <- as.matrix(dist(reducedDim))
    cellPairwiseDistances(cds) <- dp
    gp <- graph.adjacency(dp, mode = "undirected", 
                          weighted = TRUE)
    dp_mst <- minimum.spanning.tree(gp)
    minSpanningTree(cds) <- dp_mst
    cds@dim_reduce_type <- "function_passed"
  }
  else {
    reduction_method <- match.arg(reduction_method)
    if (reduction_method == "tSNE") {
      if (verbose) 
        message("Remove noise by PCA ...")
      if ("num_dim" %in% names(extra_arguments)) {
        num_dim <- extra_arguments$num_dim
      }
      else {
        num_dim <- 50
      }
      FM <- (FM)
      irlba_res <- prcomp_irlba(t(FM), n = min(num_dim, 
                                               min(dim(FM)) - 1), center = TRUE, scale. = TRUE)
      irlba_pca_res <- irlba_res$x
      topDim_pca <- irlba_pca_res
      if (verbose) 
        message("Reduce dimension by tSNE ...")
      tsne_res <- Rtsne::Rtsne(as.matrix(topDim_pca), dims = max_components, 
                               pca = F, ...)
      tsne_data <- tsne_res$Y[, 1:max_components]
      row.names(tsne_data) <- colnames(tsne_data)
      reducedDimA(cds) <- t(tsne_data)
      cds@auxClusteringData[["tSNE"]]$pca_components_used <- num_dim
      cds@auxClusteringData[["tSNE"]]$reduced_dimension <- t(topDim_pca)
      cds@dim_reduce_type <- "tSNE"
    }
    else if (reduction_method == "ICA") {
      if (verbose) 
        message("Reducing to independent components")
      init_ICA <- ica_helper(Matrix::t(FM), max_components, 
                             use_irlba = TRUE, ...)
      x_pca <- Matrix::t(Matrix::t(FM) %*% init_ICA$K)
      W <- Matrix::t(init_ICA$W)
      weights <- W
      A <- Matrix::t(solve(weights) %*% Matrix::t(init_ICA$K))
      colnames(A) <- colnames(weights)
      rownames(A) <- rownames(FM)
      S <- weights %*% x_pca
      rownames(S) <- colnames(weights)
      colnames(S) <- colnames(FM)
      reducedDimW(cds) <- as.matrix(W)
      reducedDimA(cds) <- as.matrix(A)
      reducedDimS(cds) <- as.matrix(S)
      reducedDimK(cds) <- as.matrix(init_ICA$K)
      adjusted_S <- Matrix::t(reducedDimS(cds))
      dp <- as.matrix(dist(adjusted_S))
      cellPairwiseDistances(cds) <- dp
      gp <- graph.adjacency(dp, mode = "undirected", 
                            weighted = TRUE)
      dp_mst <- minimum.spanning.tree(gp)
      minSpanningTree(cds) <- dp_mst
      cds@dim_reduce_type <- "ICA"
    }
    else if (reduction_method == "DDRTree") {
      if (verbose) 
        message("Learning principal graph with DDRTree")
      if (auto_param_selection & ncol(cds) >= 100) {
        if ("ncenter" %in% names(extra_arguments)) 
          ncenter <- extra_arguments$ncenter
        else ncenter <- cal_ncenter(ncol(FM))
        ddr_args <- c(list(X = FM, dimensions = max_components, 
                           ncenter = ncenter, verbose = verbose), extra_arguments[names(extra_arguments) %in% 
                                                                                    c("initial_method", "maxIter", 
                                                                                      "sigma", "lambda", "param.gamma", 
                                                                                      "tol")])
        ddrtree_res <- do.call(DDRTree, ddr_args)
      }
      else {
        ddrtree_res <- DDRTree(FM, max_components, verbose = verbose, 
                               ...)
      }
      if (ncol(ddrtree_res$Y) == ncol(cds)) 
        colnames(ddrtree_res$Y) <- colnames(FM)
      else colnames(ddrtree_res$Y) <- paste("Y_", 
                                            1:ncol(ddrtree_res$Y), sep = "")
      colnames(ddrtree_res$Z) <- colnames(FM)
      reducedDimW(cds) <- ddrtree_res$W
      reducedDimS(cds) <- ddrtree_res$Z
      reducedDimK(cds) <- ddrtree_res$Y
      cds@auxOrderingData[["DDRTree"]]$objective_vals <- ddrtree_res$objective_vals
      adjusted_K <- Matrix::t(reducedDimK(cds))
      dp <- as.matrix(dist(adjusted_K))
      cellPairwiseDistances(cds) <- dp
      gp <- graph.adjacency(dp, mode = "undirected", 
                            weighted = TRUE)
      dp_mst <- minimum.spanning.tree(gp)
      minSpanningTree(cds) <- dp_mst
      cds@dim_reduce_type <- "DDRTree"
      cds <- findNearestPointOnMST(cds)
    }
    else {
      stop("Error: unrecognized dimensionality reduction method")
    }
  }
  cds
}

1.1 cal_ncenter

In [ ]:
function(ncells, ncells_limit = 100){
    round(2 * ncells_limit * log(ncells)/ (log(ncells) + log(ncells_limit)))
}

1.2 normalize_expr_data

In [ ]:
function(cds, root_cell, verbose=T)
{
    
    dp <- cellPairwiseDistances(cds)
    dp_mst <- minSpanningTree(cds)
    
    curr_state <- 1
    
    res <- list(subtree = dp_mst, root = root_cell)
    
    states = rep(1, ncol(dp))
    names(states) <- V(dp_mst)$name
    
    pseudotimes = rep(0, ncol(dp))
    names(pseudotimes) <- V(dp_mst)$name
    
    parents = rep(NA, ncol(dp))
    names(parents) <- V(dp_mst)$name
    
    mst_traversal <- graph.dfs(dp_mst,
                               root=root_cell,
                               neimode = "all",
                               unreachable=FALSE,
                               father=TRUE)
    mst_traversal$father <- as.numeric(mst_traversal$father)
    curr_state <- 1
    
    for (i in 1:length(mst_traversal$order)){
        curr_node = mst_traversal$order[i]
        curr_node_name = V(dp_mst)[curr_node]$name
        
        if (is.na(mst_traversal$father[curr_node]) == FALSE){
            parent_node = mst_traversal$father[curr_node]
            parent_node_name = V(dp_mst)[parent_node]$name
            parent_node_pseudotime = pseudotimes[parent_node_name]
            parent_node_state = states[parent_node_name]
            curr_node_pseudotime = parent_node_pseudotime + dp[curr_node_name, parent_node_name]
            if (degree(dp_mst, v=parent_node_name) > 2){
                curr_state <- curr_state + 1
            }
        }else{
            parent_node = NA
            parent_node_name = NA
            curr_node_pseudotime = 0
        }
        
        curr_node_state = curr_state
        pseudotimes[curr_node_name] <- curr_node_pseudotime
        states[curr_node_name] <- curr_node_state
        parents[curr_node_name] <- parent_node_name
    }
    
    ordering_df <- data.frame(sample_name = names(states),
                              cell_state = factor(states),
                              pseudo_time = as.vector(pseudotimes),
                              parent = parents)
    row.names(ordering_df) <- ordering_df$sample_name
    # ordering_df <- plyr::arrange(ordering_df, pseudo_time)
    return(ordering_df)
}
function(cds, root_cell, verbose=T)
{
    
    dp <- cellPairwiseDistances(cds)
    dp_mst <- minSpanningTree(cds)
    
    curr_state <- 1
    
    res <- list(subtree = dp_mst, root = root_cell)
    
    states = rep(1, ncol(dp))
    names(states) <- V(dp_mst)$name
    
    pseudotimes = rep(0, ncol(dp))
    names(pseudotimes) <- V(dp_mst)$name
    
    parents = rep(NA, ncol(dp))
    names(parents) <- V(dp_mst)$name
    
    mst_traversal <- graph.dfs(dp_mst,
                               root=root_cell,
                               neimode = "all",
                               unreachable=FALSE,
                               father=TRUE)
    mst_traversal$father <- as.numeric(mst_traversal$father)
    curr_state <- 1
    
    for (i in 1:length(mst_traversal$order)){
        curr_node = mst_traversal$order[i]
        curr_node_name = V(dp_mst)[curr_node]$name
        
        if (is.na(mst_traversal$father[curr_node]) == FALSE){
            parent_node = mst_traversal$father[curr_node]
            parent_node_name = V(dp_mst)[parent_node]$name
            parent_node_pseudotime = pseudotimes[parent_node_name]
            parent_node_state = states[parent_node_name]
            curr_node_pseudotime = parent_node_pseudotime + dp[curr_node_name, parent_node_name]
            if (degree(dp_mst, v=parent_node_name) > 2){
                curr_state <- curr_state + 1
            }
        }else{
            parent_node = NA
            parent_node_name = NA
            curr_node_pseudotime = 0
        }
        
        curr_node_state = curr_state
        pseudotimes[curr_node_name] <- curr_node_pseudotime
        states[curr_node_name] <- curr_node_state
        parents[curr_node_name] <- parent_node_name
    }
    
    ordering_df <- data.frame(sample_name = names(states),
                              cell_state = factor(states),
                              pseudo_time = as.vector(pseudotimes),
                              parent = parents)
    row.names(ordering_df) <- ordering_df$sample_name
    # ordering_df <- plyr::arrange(ordering_df, pseudo_time)
    return(ordering_df)
}

2. orderCells

In [ ]:
library(igraph)

In [ ]:
function (cds, root_state = NULL, num_paths = NULL, reverse = NULL) 
{
  if (class(cds)[1] != "CellDataSet") {
    stop("Error cds is not of type 'CellDataSet'")
  }
  if (is.null(cds@dim_reduce_type)) {
    stop("Error: dimensionality not yet reduced. Please call reduceDimension() before calling this function.")
  }
  if (any(c(length(cds@reducedDimS) == 0, length(cds@reducedDimK) == 
            0))) {
    stop("Error: dimension reduction didn't prodvide correct results. Please check your reduceDimension() step and ensure correct dimension reduction are performed before calling this function.")
  }
  root_cell <- select_root_cell(cds, root_state, reverse)
  cds@auxOrderingData <- new.env(hash = TRUE)
  if (cds@dim_reduce_type == "ICA") {
    if (is.null(num_paths)) {
      num_paths = 1
    }
    adjusted_S <- t(cds@reducedDimS)
    dp <- as.matrix(dist(adjusted_S))
    cellPairwiseDistances(cds) <- as.matrix(dist(adjusted_S))
    gp <- graph.adjacency(dp, mode = "undirected", 
                          weighted = TRUE)
    dp_mst <- minimum.spanning.tree(gp)
    minSpanningTree(cds) <- dp_mst
    next_node <<- 0
    res <- pq_helper(dp_mst, use_weights = FALSE, root_node = root_cell)
    cds@auxOrderingData[[cds@dim_reduce_type]]$root_cell <- root_cell
    order_list <- extract_good_branched_ordering(res$subtree, 
                                                 res$root, cellPairwiseDistances(cds), num_paths, 
                                                 FALSE)
    cc_ordering <- order_list$ordering_df
    row.names(cc_ordering) <- cc_ordering$sample_name
    minSpanningTree(cds) <- as.undirected(order_list$cell_ordering_tree)
    pData(cds)$Pseudotime <- cc_ordering[row.names(pData(cds)), 
                                         ]$pseudo_time
    pData(cds)$State <- cc_ordering[row.names(pData(cds)), 
                                    ]$cell_state
    mst_branch_nodes <- V(minSpanningTree(cds))[which(degree(minSpanningTree(cds)) > 
                                                        2)]$name
    minSpanningTree(cds) <- dp_mst
    cds@auxOrderingData[[cds@dim_reduce_type]]$cell_ordering_tree <- as.undirected(order_list$cell_ordering_tree)
  }
  else if (cds@dim_reduce_type == "DDRTree") {
    if (is.null(num_paths) == FALSE) {
      message("Warning: num_paths only valid for method 'ICA' in reduceDimension()")
    }
    cc_ordering <- extract_ddrtree_ordering(cds, root_cell)
    pData(cds)$Pseudotime <- cc_ordering[row.names(pData(cds)), 
                                         ]$pseudo_time
    K_old <- reducedDimK(cds)
    old_dp <- cellPairwiseDistances(cds)
    old_mst <- minSpanningTree(cds)
    old_A <- reducedDimA(cds)
    old_W <- reducedDimW(cds)
    cds <- project2MST(cds, project_point_to_line_segment)
    minSpanningTree(cds) <- cds@auxOrderingData[[cds@dim_reduce_type]]$pr_graph_cell_proj_tree
    root_cell_idx <- which(V(old_mst)$name == root_cell, 
                           arr.ind = T)
    cells_mapped_to_graph_root <- which(cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex == 
                                          root_cell_idx)
    if (length(cells_mapped_to_graph_root) == 0) {
      cells_mapped_to_graph_root <- root_cell_idx
    }
    cells_mapped_to_graph_root <- V(minSpanningTree(cds))[cells_mapped_to_graph_root]$name
    tip_leaves <- names(which(degree(minSpanningTree(cds)) == 
                                1))
    root_cell <- cells_mapped_to_graph_root[cells_mapped_to_graph_root %in% 
                                              tip_leaves][1]
    if (is.na(root_cell)) {
      root_cell <- select_root_cell(cds, root_state, reverse)
    }
    cds@auxOrderingData[[cds@dim_reduce_type]]$root_cell <- root_cell
    cc_ordering_new_pseudotime <- extract_ddrtree_ordering(cds, 
                                                           root_cell)
    pData(cds)$Pseudotime <- cc_ordering_new_pseudotime[row.names(pData(cds)), 
                                                        ]$pseudo_time
    if (is.null(root_state) == TRUE) {
      closest_vertex <- cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex
      pData(cds)$State <- cc_ordering[closest_vertex[, 
                                                     1], ]$cell_state
    }
    reducedDimK(cds) <- K_old
    cellPairwiseDistances(cds) <- old_dp
    minSpanningTree(cds) <- old_mst
    reducedDimA(cds) <- old_A
    reducedDimW(cds) <- old_W
    mst_branch_nodes <- V(minSpanningTree(cds))[which(degree(minSpanningTree(cds)) > 
                                                        2)]$name
  }
  else if (cds@dim_reduce_type == "SimplePPT") {
    if (is.null(num_paths) == FALSE) {
      message("Warning: num_paths only valid for method 'ICA' in reduceDimension()")
    }
    cc_ordering <- extract_ddrtree_ordering(cds, root_cell)
    pData(cds)$Pseudotime <- cc_ordering[row.names(pData(cds)), 
                                         ]$pseudo_time
    pData(cds)$State <- cc_ordering[row.names(pData(cds)), 
                                    ]$cell_state
    mst_branch_nodes <- V(minSpanningTree(cds))[which(degree(minSpanningTree(cds)) > 
                                                        2)]$name
  }
  cds@auxOrderingData[[cds@dim_reduce_type]]$branch_points <- mst_branch_nodes
  cds
}

2.1 select_root_cell

In [ ]:
select_root_cell<-function(cds, root_state=NULL, reverse=FALSE){
    if (is.null(root_state) == FALSE) {
        if (is.null(pData(cds)$State)){
            stop("Error: State has not yet been set. Please call orderCells() without specifying root_state, then try this call again.")
        }
        # FIXME: Need to gaurd against the case when the supplied root state isn't actually a terminal state in the tree.
        root_cell_candidates <- subset(pData(cds), State == root_state)
        if (nrow(root_cell_candidates) == 0){
            stop(paste("Error: no cells for State =", root_state))
        }
        
        # build a local MST to find a good root cell for this state
        dp <- as.matrix(dist(t(reducedDimS(cds)[,row.names(root_cell_candidates)])))
        gp <- graph.adjacency(dp, mode = "undirected", weighted = TRUE)
        dp_mst <- minimum.spanning.tree(gp)
        
        # Make sure to use the real MST here
        tip_leaves <- names(which(degree(minSpanningTree(cds)) == 1))
        #root_cell_candidates <- root_cell_candidates[row.names(root_cell_candidates) %in% tip_leaves,]
        #sg <- make_ego_graph(dp_mst, nodes=row.names(root_cell_candidates))[[1]]
        
        # if(length(intersect(tip_leaves, row.names(root_cell_candidates))) == 0){
        #   stop(paste("Error: no valid root cells for State =", root_state))
        # }
        
        diameter <- get.diameter(dp_mst)
        
        if (length(diameter) == 0){
            stop(paste("Error: no valid root cells for State =", root_state))
        }
        
        #root_cell = names(diameter)[tip_leaves %in% names(diameter)]
        root_cell_candidates <- root_cell_candidates[names(diameter),]
        if (is.null(cds@auxOrderingData[[cds@dim_reduce_type]]$root_cell) == FALSE &&
            pData(cds)[cds@auxOrderingData[[cds@dim_reduce_type]]$root_cell,]$State == root_state){
            root_cell <- row.names(root_cell_candidates)[which(root_cell_candidates$Pseudotime == min(root_cell_candidates$Pseudotime))]
        }else{
            root_cell <- row.names(root_cell_candidates)[which(root_cell_candidates$Pseudotime == max(root_cell_candidates$Pseudotime))]
        }
        if (length(root_cell) > 1)
            root_cell <- root_cell[1]
        
        # If we used DDRTree, we need to go from this actual cell to the nearst
        # point on the principal graph
        if (cds@dim_reduce_type == "DDRTree"){
            #root_cell_idx <- which(V(minSpanningTree(cds))$name == root_cell, arr.ind=T)
            graph_point_for_root_cell <- cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex[root_cell,]
            root_cell = V(minSpanningTree(cds))[graph_point_for_root_cell]$name
        }
        
    }else{
        if (is.null(minSpanningTree(cds))){
            stop("Error: no spanning tree found for CellDataSet object. Please call reduceDimension before calling orderCells()")
        }
        diameter <- get.diameter(minSpanningTree(cds))
        if (is.null(reverse) == FALSE && reverse == TRUE){
            root_cell = names(diameter[length(diameter)])
        } else {
            root_cell = names(diameter[1])
        }
    }
    return(root_cell)
}

2.2 extract_ddrtree_ordering

In [ ]:
extract_ddrtree_ordering<-function(cds, root_cell, verbose=T)
{
    
    dp <- cellPairwiseDistances(cds)
    dp_mst <- minSpanningTree(cds)
    
    curr_state <- 1
    
    res <- list(subtree = dp_mst, root = root_cell)
    
    states = rep(1, ncol(dp))
    names(states) <- V(dp_mst)$name
    
    pseudotimes = rep(0, ncol(dp))
    names(pseudotimes) <- V(dp_mst)$name
    
    parents = rep(NA, ncol(dp))
    names(parents) <- V(dp_mst)$name
    
    mst_traversal <- graph.dfs(dp_mst,
                               root=root_cell,
                               neimode = "all",
                               unreachable=FALSE,
                               father=TRUE)
    mst_traversal$father <- as.numeric(mst_traversal$father)
    curr_state <- 1
    
    for (i in 1:length(mst_traversal$order)){
        curr_node = mst_traversal$order[i]
        curr_node_name = V(dp_mst)[curr_node]$name
        
        if (is.na(mst_traversal$father[curr_node]) == FALSE){
            parent_node = mst_traversal$father[curr_node]
            parent_node_name = V(dp_mst)[parent_node]$name
            parent_node_pseudotime = pseudotimes[parent_node_name]
            parent_node_state = states[parent_node_name]
            curr_node_pseudotime = parent_node_pseudotime + dp[curr_node_name, parent_node_name]
            if (degree(dp_mst, v=parent_node_name) > 2){
                curr_state <- curr_state + 1
            }
        }else{
            parent_node = NA
            parent_node_name = NA
            curr_node_pseudotime = 0
        }
        
        curr_node_state = curr_state
        pseudotimes[curr_node_name] <- curr_node_pseudotime
        states[curr_node_name] <- curr_node_state
        parents[curr_node_name] <- parent_node_name
    }
    
    ordering_df <- data.frame(sample_name = names(states),
                              cell_state = factor(states),
                              pseudo_time = as.vector(pseudotimes),
                              parent = parents)
    row.names(ordering_df) <- ordering_df$sample_name
    # ordering_df <- plyr::arrange(ordering_df, pseudo_time)
    return(ordering_df)
}

2.3 project2MST

In [ ]:
project2MST<-function(cds, Projection_Method){
    dp_mst <- minSpanningTree(cds)
    Z <- reducedDimS(cds)
    Y <- reducedDimK(cds)
    
    cds <- findNearestPointOnMST(cds)
    closest_vertex <- cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex
    
    #closest_vertex <- as.vector(closest_vertex)
    closest_vertex_names <- colnames(Y)[closest_vertex]
    closest_vertex_df <- as.matrix(closest_vertex)
    row.names(closest_vertex_df) <- row.names(closest_vertex)
    #closest_vertex_names <- as.vector(closest_vertex)
    
    tip_leaves <- names(which(degree(dp_mst) == 1))
    
    if(!is.function(Projection_Method)) {
        P <- Y[, closest_vertex]
    }
    else{
        P <- matrix(rep(0, length(Z)), nrow = nrow(Z)) #Y
        for(i in 1:length(closest_vertex)) {
            neighbors <- names(V(dp_mst) [ suppressWarnings(nei(closest_vertex_names[i], mode="all")) ])
            projection <- NULL
            distance <- NULL
            Z_i <- Z[, i]
            
            for(neighbor in neighbors) {
                if(closest_vertex_names[i] %in% tip_leaves) {
                    tmp <- projPointOnLine(Z_i, Y[, c(closest_vertex_names[i], neighbor)]) #projPointOnLine: always perform orthogonal projection to the line
                }
                else {
                    tmp <- Projection_Method(Z_i, Y[, c(closest_vertex_names[i], neighbor)])
                }
                projection <- rbind(projection, tmp)
                distance <- c(distance, dist(rbind(Z_i, tmp)))
            }
            if(class(projection) != 'matrix')
                projection <- as.matrix(projection)
            P[, i] <- projection[which(distance == min(distance))[1], ] #use only the first index to avoid assignment error
        }
    }
    # tip_leaves <- names(which(degree(minSpanningTree(cds)) == 1))
    
    colnames(P) <- colnames(Z)
    
    #reducedDimK(cds) <- P
    dp <- as.matrix(dist(t(P)))
    #dp <- as.matrix(dist(t(reducedDimS(cds))))
    
    min_dist = min(dp[dp!=0])
    #dp[dp == 0] <- min_dist
    dp <- dp + min_dist #to avoid exact Pseudotime for a lot cells
    diag(dp) <- 0
    
    cellPairwiseDistances(cds) <- dp
    gp <- graph.adjacency(dp, mode = "undirected", weighted = TRUE)
    dp_mst <- minimum.spanning.tree(gp)
    
    cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_tree <- dp_mst
    cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_dist <- P #dp, P projection point not output
    cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex <- closest_vertex_df #as.matrix(closest_vertex)
    
    cds
}

2.3.1 project_point_to_line_segment

In [ ]:
project_point_to_line_segment<-function(p, df){
    # returns q the closest point to p on the line segment from A to B
    A <- df[, 1]
    B <- df[, 2]
    # vector from A to B
    AB <- (B-A)
    # squared distance from A to B
    AB_squared = sum(AB^2)
    if(AB_squared == 0) {
        # A and B are the same point
        q <- A
    }
    else {
        # vector from A to p
        Ap <- (p-A)
        # from http://stackoverflow.com/questions/849211/
        # Consider the line extending the segment, parameterized as A + t (B - A)
        # We find projection of point p onto the line.
        # It falls where t = [(p-A) . (B-A)] / |B-A|^2
        # t <- max(0, min(1, sum(Ap * AB) / AB_squared))
        t <- sum(Ap * AB) / AB_squared
        
        if (t < 0.0) {
            # "Before" A on the line, just return A
            q <- A
        }
        else if (t > 1.0) {
            # "After" B on the line, just return B
            q <- B
        }
        else {
            # projection lines "inbetween" A and B on the line
            q <- A + t * AB#
        }
    }
    return(q)
}

2.3.2 findNearestPointOnMST

In [ ]:
findNearestPointOnMST<-function(cds){
    dp_mst <- minSpanningTree(cds)
    Z <- reducedDimS(cds)
    Y <- reducedDimK(cds)
    
    tip_leaves <- names(which(degree(dp_mst) == 1))
    
    distances_Z_to_Y <- proxy::dist(t(Z), t(Y))
    closest_vertex <- apply(distances_Z_to_Y, 1, function(z) { which ( z == min(z) )[1] } )
    #closest_vertex <- which(distance_to_closest == min(distance_to_closest))
    
    #closest_vertex <- as.vector(closest_vertex)
    closest_vertex_names <- colnames(Y)[closest_vertex]
    closest_vertex_df <- as.matrix(closest_vertex) #index on Z
    row.names(closest_vertex_df) <- names(closest_vertex) #original cell names for projection
    
    cds@auxOrderingData[["DDRTree"]]$pr_graph_cell_proj_closest_vertex <- closest_vertex_df #as.matrix(closest_vertex)
    cds
}

2.3.3 projPointOnLine

In [ ]:
projPointOnLine<-function(point, line) {
    ap <- point - line[, 1]
    ab <- line[, 2] - line[, 1]
    
    res <- line[, 1] + c((ap %*% ab) / (ab %*% ab)) * ab
    return(res)
}

3.differentialGeneTest

In [ ]:
differentialGeneTest<-function (cds, fullModelFormulaStr = "~sm.ns(Pseudotime, df=3)", 
    reducedModelFormulaStr = "~1", relative_expr = TRUE, 
    cores = 1, verbose = FALSE) 
{
    status <- NA
    if (class(cds)[1] != "CellDataSet") {
        stop("Error cds is not of type 'CellDataSet'")
    }
    all_vars <- c(all.vars(formula(fullModelFormulaStr)), all.vars(formula(reducedModelFormulaStr)))
    pd <- pData(cds)
    for (i in all_vars) {
        x <- pd[, i]
        if (any((c(Inf, NaN, NA) %in% x))) {
            stop("Error: Inf, NaN, or NA values were located in pData of cds in columns mentioned in model terms")
        }
    }
    if (relative_expr && cds@expressionFamily@vfamily %in% c("negbinomial", 
        "negbinomial.size")) {
        if (is.null(sizeFactors(cds)) || sum(is.na(sizeFactors(cds)))) {
            stop("Error: to call this function with relative_expr==TRUE, you must first call estimateSizeFactors() on the CellDataSet.")
        }
    }
    if (cores > 1) {
        diff_test_res <- mcesApply(cds, 1, diff_test_helper, 
            c("BiocGenerics", "VGAM", "Matrix"), 
            cores = cores, fullModelFormulaStr = fullModelFormulaStr, 
            reducedModelFormulaStr = reducedModelFormulaStr, 
            expressionFamily = cds@expressionFamily, relative_expr = relative_expr, 
            disp_func = cds@dispFitInfo[["blind"]]$disp_func, 
            verbose = verbose)
        diff_test_res
    }
    else {
        diff_test_res <- smartEsApply(cds, 1, diff_test_helper, 
            convert_to_dense = TRUE, fullModelFormulaStr = fullModelFormulaStr, 
            reducedModelFormulaStr = reducedModelFormulaStr, 
            expressionFamily = cds@expressionFamily, relative_expr = relative_expr, 
            disp_func = cds@dispFitInfo[["blind"]]$disp_func, 
            verbose = verbose)
        diff_test_res
    }
    diff_test_res <- do.call(rbind.data.frame, diff_test_res)
    diff_test_res$qval <- 1
    diff_test_res$qval[which(diff_test_res$status == "OK")] <- p.adjust(subset(diff_test_res, 
        status == "OK")[, "pval"], method = "BH")
    diff_test_res <- merge(diff_test_res, fData(cds), by = "row.names")
    row.names(diff_test_res) <- diff_test_res[, 1]
    diff_test_res[, 1] <- NULL
    diff_test_res[row.names(cds), ]
}

In [ ]:
smartEsApply <- function(X, MARGIN, FUN, convert_to_dense, ...) {
  parent <- environment(FUN)
  if (is.null(parent))
    parent <- emptyenv()
  e1 <- new.env(parent=parent)
  multiassign(names(pData(X)), pData(X), envir=e1)
  environment(FUN) <- e1
  
  if (isSparseMatrix(exprs(X))){
    res <- sparseApply(exprs(X), MARGIN, FUN, convert_to_dense, ...)
  }else{
    res <- apply(exprs(X), MARGIN, FUN, ...)
  }
  
  if (MARGIN == 1)
  {
    names(res) <- row.names(X)
  }else{
    names(res) <- colnames(X)
  }

  res
}

In [ ]:
diff_test_helper <- function(x, 
                             fullModelFormulaStr, 
                             reducedModelFormulaStr, 
                             expressionFamily, 
                             relative_expr,
                             weights,
                             disp_func=NULL,
                             verbose=FALSE
                             ){ 
  
  reducedModelFormulaStr <- paste("f_expression", reducedModelFormulaStr, sep="")
  fullModelFormulaStr <- paste("f_expression", fullModelFormulaStr, sep="")
  
  x_orig <- x
  disp_guess <- 0
  
  if (expressionFamily@vfamily %in% c("negbinomial", "negbinomial.size")){
    if (relative_expr == TRUE)
    {
      x <- x / Size_Factor
    }
    f_expression <- round(x)
    if (is.null(disp_func) == FALSE){
      disp_guess <- calculate_NB_dispersion_hint(disp_func, round(x_orig))
      if (is.null(disp_guess) == FALSE && disp_guess > 0 && is.na(disp_guess) == FALSE  ) {
        # FIXME: In theory, we could lose some user-provided parameters here
        # e.g. if users supply zero=NULL or something. 
        if (expressionFamily@vfamily == "negbinomial")
          expressionFamily <- negbinomial(isize=1/disp_guess)
        else
          expressionFamily <- negbinomial.size(size=1/disp_guess)
      }
    }
  }else if (expressionFamily@vfamily %in% c("uninormal")){
    f_expression <- x
  }else if (expressionFamily@vfamily %in% c("binomialff")){
    f_expression <- x
    #f_expression[f_expression > 1] <- 1
  }else{
    f_expression <- log10(x)
  }
  
  test_res <- tryCatch({
    if (expressionFamily@vfamily %in% c("binomialff")){
      if (verbose){
        full_model_fit <- VGAM::vglm(as.formula(fullModelFormulaStr), epsilon=1e-1, family=expressionFamily)
        reduced_model_fit <- VGAM::vglm(as.formula(reducedModelFormulaStr), epsilon=1e-1, family=expressionFamily)                         
      }else{
        full_model_fit <- suppressWarnings(VGAM::vglm(as.formula(fullModelFormulaStr), epsilon=1e-1, family=expressionFamily))
        reduced_model_fit <- suppressWarnings(VGAM::vglm(as.formula(reducedModelFormulaStr), epsilon=1e-1, family=expressionFamily))                    
      }
    }else{
      if (verbose){
        full_model_fit <- VGAM::vglm(as.formula(fullModelFormulaStr), epsilon=1e-1, family=expressionFamily)
        reduced_model_fit <- VGAM::vglm(as.formula(reducedModelFormulaStr), epsilon=1e-1, family=expressionFamily)                         
      }else{
        full_model_fit <- suppressWarnings(VGAM::vglm(as.formula(fullModelFormulaStr), epsilon=1e-1, family=expressionFamily))
        reduced_model_fit <- suppressWarnings(VGAM::vglm(as.formula(reducedModelFormulaStr), epsilon=1e-1, family=expressionFamily))                    
      }
    }

    #print(full_model_fit)
    #print(coef(reduced_model_fit))
    compareModels(list(full_model_fit), list(reduced_model_fit))
  }, 
  #warning = function(w) { FM_fit },
  error = function(e) { 
    if(verbose)
      print (e);
      data.frame(status = "FAIL", family=expressionFamily@vfamily, pval=1.0, qval=1.0)
    #data.frame(status = "FAIL", pval=1.0) 
  }
  )
  test_res
}

In [ ]:
isSparseMatrix <- function(x){
  class(x) %in% c("dgCMatrix", "dgTMatrix")
}

# Estimate size factors for each column, given a sparseMatrix from the Matrix
# package
#' @import slam
#' @importFrom stats median
estimateSizeFactorsForSparseMatrix <- function(counts, 
                                               locfunc = median, 
                                               round_exprs=TRUE, 
                                               method="mean-geometric-mean-total"){
  CM <- counts
  if (round_exprs)
    CM <- round(CM)
  CM <- asSlamMatrix(CM)
  
  if (method == "weighted-median"){

    log_medians <- rowapply_simple_triplet_matrix(CM, function(cell_expr) { 
      log(locfunc(cell_expr))
    })
    
    weights <- rowapply_simple_triplet_matrix(CM, function(cell_expr) {
      num_pos <- sum(cell_expr > 0)
      num_pos / length(cell_expr)
    })
    
    sfs <- colapply_simple_triplet_matrix(CM, function(cnts) {
      norm_cnts <-  weights * (log(cnts) -  log_medians)
      norm_cnts <- norm_cnts[is.nan(norm_cnts) == FALSE]
      norm_cnts <- norm_cnts[is.finite(norm_cnts)]
      #print (head(norm_cnts))
      exp( mean(norm_cnts) )
    })
  }else if (method == "median-geometric-mean"){
    log_geo_means <- rowapply_simple_triplet_matrix(CM, function(x) { mean(log(CM)) })
    
    sfs <- colapply_simple_triplet_matrix(CM, function(cnts) {
      norm_cnts <- log(cnts) -  log_geo_means
      norm_cnts <- norm_cnts[is.nan(norm_cnts) == FALSE]
      norm_cnts <- norm_cnts[is.finite(norm_cnts)]
      #print (head(norm_cnts))
      exp( locfunc( norm_cnts ))
    })
  }else if(method == "median"){
    stop("Error: method 'median' not yet supported for sparse matrices")
  }else if(method == 'mode'){
    stop("Error: method 'mode' not yet supported for sparse matrices")
  }else if(method == 'geometric-mean-total') {
    cell_total <- col_sums(CM)
    sfs <- log(cell_total) / mean(log(cell_total))
  }else if(method == 'mean-geometric-mean-total') {
    cell_total <- col_sums(CM)
    sfs <- cell_total / exp(mean(log(cell_total)))
  } 
  
  sfs[is.na(sfs)] <- 1 
  sfs   
}
